<a href="https://colab.research.google.com/github/rcelebi/iswc2020-smarttask/blob/master/BERT_type_pred_dbpedia_top_level.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import os
path_to_ipynb = '/content/drive/My Drive/iswc-ammar/shervin'
# change directory to where autoregq.py, src, and data are
os.chdir(path_to_ipynb)

In [3]:
import pandas as pd
import numpy as np
dbpedia_df = pd.read_json('data/smarttask_dbpedia_train.json')

In [4]:
test_df= pd.read_json('data/smarttask_dbpedia_test.json')

In [5]:
test_df.head()

,id,question,category,type
0,dbpedia_16015,How many ingredients are in the grain} ?,literal,[number]
1,dbpedia_3885,Is the case fatality rate of Fournier gangrene...,boolean,[boolean]
2,dbpedia_12907,Does the shelf life of spinach equal 8?,boolean,[boolean]
3,dbpedia_7955,What sound does a pig make in the French langu...,literal,[string]
4,dbpedia_2376,When was Fergie completed his record label in ...,literal,[date]


In [6]:
answer_types = {}
for answers in dbpedia_df.type:
    for a in answers:
        if a in answer_types:
            answer_types[a]+=1
        else:
            answer_types[a]=1

def multitype_to_freqtype(type_list):
    for t in sorted(sorted_types, key=lambda item: item[1]):
        if sorted_types[t] < 1000: 
          if t in type_list:
              return t

    for t in sorted(sorted_types, key=lambda item: item[1]):
          if t in type_list:
              return t
    

def types_to_bottom_type(type_list):
    if len(type_list) == 0:
      return None 
    return type_list[0]


def types_to_top_type(type_list):
    if len(type_list) == 0:
      return None 
    return type_list[-1]
            
answ_df =pd.DataFrame.from_dict(answer_types, orient='index', columns=['Freq'])
sorted_types = answ_df.sort_values(by='Freq', ascending=False).to_dict()['Freq']

dbpedia_df['frequent_type'] =dbpedia_df.type.apply(multitype_to_freqtype)
dbpedia_df['bottom_level_type'] =dbpedia_df.type.apply(types_to_bottom_type)
dbpedia_df['top_level_type'] =dbpedia_df.type.apply(types_to_top_type)

In [7]:
test_df['frequent_type'] =test_df.type.apply(multitype_to_freqtype)
test_df['bottom_level_type'] =test_df.type.apply(types_to_bottom_type)
test_df['top_level_type'] =test_df.type.apply(types_to_top_type)

In [8]:
dbpedia_df= dbpedia_df[dbpedia_df.category.notna()]
test_df= test_df[test_df.category.notna()]

In [9]:
dbpedia_df= dbpedia_df[dbpedia_df['frequent_type'].notna()]
test_df= test_df[test_df['frequent_type'].notna()]

In [10]:
dbpedia_df.dropna( subset=['question'], inplace=True)
test_df.dropna( subset=['question'], inplace=True)

In [11]:
dbpedia_df.head()

,id,question,category,type,frequent_type,bottom_level_type,top_level_type
0,dbpedia_1177,Was Jacqueline Kennedy Onassis a follower of M...,boolean,[boolean],boolean,boolean,boolean
1,dbpedia_14427,What is the name of the opera based on Twelfth...,resource,"[dbo:Opera, dbo:MusicalWork, dbo:Work]",dbo:Work,dbo:Opera,dbo:Work
2,dbpedia_16615,When did Lena Horne receive the Grammy Award f...,literal,[date],date,date,date
3,dbpedia_23480,Do Prince Harry and Prince William have the sa...,boolean,[boolean],boolean,boolean,boolean
4,dbpedia_3681,What is the subsidiary company working for Leo...,resource,"[dbo:EducationalInstitution, dbo:Organisation,...",dbo:EducationalInstitution,dbo:EducationalInstitution,dbo:Agent


In [12]:
def extend_cat(row):
  if row['category'] =='boolean':
    return row['category']
  elif row['category'] == 'literal':
    return row['frequent_type']
  else:
    return row['category']


In [13]:
dbpedia_df['extended_cat'] = dbpedia_df.apply(extend_cat, axis=1)

In [14]:
VALIDATION_SPLIT = 0.2
indicies = np.arange(dbpedia_df.shape[0])
np.random.shuffle(indicies)
num_valid_samples = int(VALIDATION_SPLIT * dbpedia_df.shape[0])
num_valid_samples

3502

In [15]:
num_test_samples=  int(num_valid_samples/2)

In [16]:
dbpedia_df=dbpedia_df.iloc[indicies]

In [17]:
train_df= dbpedia_df[:-num_valid_samples]
train_df.head()

,id,question,category,type,frequent_type,bottom_level_type,top_level_type,extended_cat
15517,dbpedia_3243,When did Keith Holyoake become the Prime Minis...,literal,[date],date,date,date,date
6693,dbpedia_4048,What are the recurring events which start with...,resource,[dbo:Event],dbo:Event,dbo:Event,dbo:Event,resource
7146,dbpedia_5945,How many destinations does Voyager 2 have?,literal,[number],number,number,number,number
16189,dbpedia_16098,What Dilwale Dulhania Le Jayenge award did Kaj...,resource,[dbo:Award],dbo:Award,dbo:Award,dbo:Award,resource
16922,dbpedia_1190,What kind of employment do the spouses of Juss...,resource,"[dbo:Profession, dbo:PersonFunction]",dbo:PersonFunction,dbo:Profession,dbo:PersonFunction,resource


In [18]:
valid_df= dbpedia_df[-num_valid_samples:]
valid_df.head()

,id,question,category,type,frequent_type,bottom_level_type,top_level_type,extended_cat
5758,dbpedia_13243,What is Costco's permanent ID number?,literal,[string],string,string,string,string
7917,dbpedia_1186,What movie was Sissy Spacek nominated for an A...,resource,"[dbo:Film, dbo:Work]",dbo:Work,dbo:Film,dbo:Work,resource
10496,dbpedia_5728,"Are there 2 parts to the work of art, ""Schwand...",boolean,[boolean],boolean,boolean,boolean,boolean
9449,dbpedia_20627,What are the relatives of the rugby player one...,resource,"[dbo:RugbyPlayer, dbo:Athlete, dbo:Person, dbo...",dbo:Athlete,dbo:RugbyPlayer,dbo:Agent,resource
16711,dbpedia_11642,Is the acetaldehyde autoignition temperature 1...,boolean,[boolean],boolean,boolean,boolean,boolean


In [19]:
len(valid_df)

3502

In [20]:
#valid_df.to_csv('/content/drive/My Drive/fastai-v3/data/smarttask/dbpedia_valid_gold.csv', index=False)

In [ ]:
#test_df= dbpedia_df[-num_test_samples:]
#test_df.head()

In [21]:
len(valid_df)

3502

## LSTM

## Most frequent/generic type prediction

In [22]:
len(dbpedia_df.top_level_type.unique())

47

In [23]:
train_res_df= train_df[train_df.category =='resource']
valid_res_df= valid_df[valid_df.category =='resource']

In [24]:
len(train_res_df), len(valid_res_df)

(7663, 1894)

In [26]:
type_hierarchy = {}
to_upper_class = {}
for i, row in dbpedia_df.type.iteritems():
    #print (row)
    types= list(row)
    for i in range(len(types)-1):
      parent = types[i]
      child = types[i+1]
      #print (parent, child) 
      if parent not in type_hierarchy:
          type_hierarchy[parent] = [child]
      else:
          type_hierarchy[parent].append(child)
          
      if child not in to_upper_class:
          to_upper_class[child] = parent

In [27]:
def to_super_class(specific_type, type_with_less_freq):
    if specific_type in type_with_less_freq:
        if specific_type in to_upper_class:
            #if to_upper_class[specific_type] == 'owl:Thing':
            #   return specific_type
            return to_upper_class[specific_type]
    return specific_type

def to_sub_class(specific_type, type_with_less_freq):
    if specific_type in type_with_less_freq:
        if specific_type in to_upper_class:
            #if to_upper_class[specific_type] == 'owl:Thing':
            #   return specific_type
            return type_hierarchy[parent]
    return specific_type

def choose_freqent_types_balancing(specific_type, iters, min_freq):
    for i in range(iters):
        specific_type_freq = specific_type.value_counts()
        type_with_less_freq = list(specific_type_freq[specific_type_freq < min_freq].keys())
        specific_type = specific_type.apply(lambda x: to_super_class(x, type_with_less_freq) )
    return specific_type

def balance_types(specific_type, min_freq, max_freq):
    # slipt to super-classes if number of 
    
    specific_type_freq = specific_type.value_counts()
    max_freq =  specific_type_freq.sum() * 2  / len(specific_type_freq)
    type_with_most_freq = list(specific_type_freq[specific_type_freq > max_freq].keys())
    type_with_less_freq = list(specific_type_freq[specific_type_freq < min_freq].keys())
    
    specific_type = specific_type.apply(lambda x: to_super_class(x, type_with_less_freq) )


def choose_freqent_types_balancing(specific_type, iters, min_freq):
    for i in range(iters):
        specific_type_freq = specific_type.value_counts()
        type_with_less_freq = list(specific_type_freq[specific_type_freq < min_freq].keys())
        specific_type = specific_type.apply(lambda x: to_super_class(x, type_with_less_freq) )
    return specific_type  

In [29]:
#train_res_df['frequent_type'] =choose_freqent_types_balancing(train_res_df['frequent_type'], 50, 200)

### BERT Text Classifier


In [30]:
import torch
from tqdm.notebook import tqdm

In [31]:
!pip install transformers

     |████████████████████████████████| 1.8MB 20.2MB/s 
     |████████████████████████████████| 3.2MB 60.3MB/s 
     |████████████████████████████████| 890kB 51.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=82ffcef2393bf9637d7242aa2991303b4013983bbc65719f711d4a7338636134
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [32]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

In [33]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [34]:
encoded_data_train = tokenizer.batch_encode_plus(
    train_res_df.question.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    valid_res_df.question.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [35]:
encoded_data_train

{'input_ids': tensor([[  101,  2054,  2024,  ...,     0,     0,     0],
        [  101,  2054, 29454,  ...,     0,     0,     0],
        [  101,  2054,  2785,  ...,     0,     0,     0],
        ...,
        [  101,  2171,  1996,  ...,     0,     0,     0],
        [  101,  2425,  2033,  ...,     0,     0,     0],
        [  101,  2054,  2003,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [38]:
target = 'top_level_type'
label_dict = {}
for index, possible_label in enumerate(sorted(dbpedia_df[dbpedia_df.category=='resource'][target].unique())):
    label_dict[possible_label] = index
label_dict


{'dbo:Activity': 0,
 'dbo:Agent': 1,
 'dbo:AnatomicalStructure': 2,
 'dbo:Award': 3,
 'dbo:BaseballTeam': 4,
 'dbo:Biomolecule': 5,
 'dbo:ChemicalSubstance': 6,
 'dbo:City': 7,
 'dbo:CollegeCoach': 8,
 'dbo:Country': 9,
 'dbo:Currency': 10,
 'dbo:Device': 11,
 'dbo:Disease': 12,
 'dbo:EthnicGroup': 13,
 'dbo:Event': 14,
 'dbo:Flag': 15,
 'dbo:Food': 16,
 'dbo:Holiday': 17,
 'dbo:HorseRace': 18,
 'dbo:Language': 19,
 'dbo:Location': 20,
 'dbo:MeanOfTransportation': 21,
 'dbo:Media': 22,
 'dbo:MedicalSpecialty': 23,
 'dbo:MountainRange': 24,
 'dbo:Museum': 25,
 'dbo:MusicalArtist': 26,
 'dbo:Name': 27,
 'dbo:OfficeHolder': 28,
 'dbo:Person': 29,
 'dbo:PersonFunction': 30,
 'dbo:PoliticalParty': 31,
 'dbo:PublicService': 32,
 'dbo:RecordLabel': 33,
 'dbo:River': 34,
 'dbo:SoccerClub': 35,
 'dbo:SoccerPlayer': 36,
 'dbo:Species': 37,
 'dbo:TimePeriod': 38,
 'dbo:TopicalConcept': 39,
 'dbo:UnitOfWork': 40,
 'dbo:University': 41,
 'dbo:Work': 42}

In [41]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(train_res_df[target].replace(label_dict).values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor( valid_res_df[target].replace(label_dict).values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [42]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [43]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [44]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [45]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')


def accuracy(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    acc =accuracy_score(labels_flat, preds_flat)
    print(f'Accuracy: {acc}\n')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [46]:
device = "cuda:0"
model = model.to(device)


In [47]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [48]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'/content/drive/MyDrive/fastai-v3/model/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    val_acc = accuracy(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')
    tqdm.write(f'Accuracy : {val_acc}')


Epoch 1
Training loss: 1.2672373253092126
Accuracy: 0.7803590285110876

Validation loss: 1.0624517343925088
F1 Score (Weighted): 0.7402352803723108
Accuracy : None



Epoch 2
Training loss: 0.7056557711036129
Accuracy: 0.8162618796198522

Validation loss: 1.019670407402718
F1 Score (Weighted): 0.8025233138199148
Accuracy : None



Epoch 3
Training loss: 0.4947369750395942
Accuracy: 0.8289334741288279

Validation loss: 0.9809819248403995
F1 Score (Weighted): 0.8185697323464676
Accuracy : None



Epoch 4
Training loss: 0.3571820069183942
Accuracy: 0.8315733896515312

Validation loss: 1.0615130136446376
F1 Score (Weighted): 0.8222228677093911
Accuracy : None



Epoch 5
Training loss: 0.27483487561481335
Accuracy: 0.8299894403379092

Validation loss: 1.0800773772417311
F1 Score (Weighted): 0.823042486727229
Accuracy : None



In [49]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)




Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [50]:
model.to(device)

model.load_state_dict(torch.load('/content/drive/MyDrive/fastai-v3/model/finetuned_BERT_epoch_5.model', map_location=torch.device('cpu')))


<All keys matched successfully>

Accuracy: 0.4258474576271186



In [52]:
test_res_df= test_df[test_df.category =='resource']

In [ ]:
test_res_df= test_res_df[test_res_df[target].isin(label_dict.keys())]

In [67]:
encoded_data_test = tokenizer.batch_encode_plus(
    test_res_df.question.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [68]:
input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor( test_res_df[target].replace(label_dict).values)

dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

In [69]:
dataloader_test = DataLoader(dataset_test, 
                                   sampler=SequentialSampler(dataset_test), 
                                   batch_size=batch_size)

In [70]:
_, predictions, true_vals = evaluate(dataloader_test)
accuracy(predictions, true_vals)

Accuracy: 0.810710498409332

